In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

from topogram_client import TopogramAPIClient

from csv import DictReader
import os
import json
import pymongo
#import musicbrainzngs
import arrow
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict
from dateutil import parser
from datetime import timedelta
import pickle
import time
from slugify import slugify

In [2]:
import colorsys
def get_N_HexCol(N=20):

    HSV_tuples = [(x*1.0/N, 0.5, 0.5) for x in xrange(N)]
    hex_out = []
    for rgb in HSV_tuples:
        rgb = map(lambda x: int(x*255),colorsys.hsv_to_rgb(*rgb))
        hex_out.append("".join(map(lambda x: chr(x).encode('hex'),rgb)))
    return hex_out

In [3]:
def create_topogram(title, nodes, edges):

    print "Creating topogram '%s'"%title

    try :
        r = topogram.create_topogram(title)
    except ValueError:
        print '> Topogram already exists'
        r = topogram.get_topogram_by_name(title)
    print r
    topogram_ID = r["data"]["_id"]
    print "topogram ID : %s"%topogram_ID

    # get and backup existing nodes and edges
    existing_nodes = topogram.get_nodes(topogram_ID)["data"]
    existing_edges = topogram.get_edges(topogram_ID)["data"]

    # clear existing graph
    if len(existing_nodes):
        topogram.delete_nodes([n["_id"] for n in existing_nodes])
        print "%s nodes deleted"%len(existing_nodes)
    if len(existing_edges):
        topogram.delete_edges([n["_id"] for n in existing_edges])
        print "%s edges deleted"%len(existing_edges)

    r = topogram.create_nodes(topogram_ID, nodes)
    print r
    print "%s nodes created."%len(r["data"])
    r = topogram.create_edges(topogram_ID, edges)
    print "%s edges created."%len(r["data"])

    print "done. Topogram has been updated. Check it at %s/topograms/%s"%(TOPOGRAM_URL, topogram_ID)



In [67]:
import math
# setup mongo
client = MongoClient()
db = client["bandstour"]
bandsintown =  db["minedArtists"]

# credentials
TOPOGRAM_URL = "https://app.topogram.io"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

# data
title = "Foo Fighters bandstour 0.1"
my_nodesdict = {}
my_nodes= []
my_edges = []
my_edgesdict = {}

# date parsing
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
DATETIME_FORMAT_FOR_MSBNZ ='%Y-%m-%d'

#retrieve trip coords for each artist
dict_coords={}
colors_for_nodes_and_edges = get_N_HexCol()

for artist in db.minedArtists.find({"totalKm": {"$gte": 2000000}}) :
 title = artist["name"]   
 if os.path.exists("./bands_dicts/"+artist["name"]+".obj")==True:
        print "SKIPPED",artist["name"]
        pass
 else:
    my_nodesdict = {}
    my_nodes= []
    my_edges = []
    my_nodesdictList=[]
    my_edgesdict = {}
    #colors_for_nodes_and_edges = get_N_HexCol()
    
    print artist["name"]
    #print artist
    dates_coords = []
    dates = artist.get("gigs")
    tours = artist.get("tours")    
    singlegigs = artist.get("singleGigs")
    dateini=0
    touring=0
    tour_indice=0
    tmpOLDgig ={}
    for tour in tours:
     tour_color = "#"+colors_for_nodes_and_edges[tour_indice % 20]
     #print tour_color
     #print tour
     gig_indice =0
     dates = tour.get("gigs")
     for date in dates:
        #print date.get("datetime")
        venue = date.get("venue")
        latitude = venue.get("latitude")
        longitude = venue.get("longitude")
        #print venue
        ###concat key
        idd= slugify(venue["name"])+str(int(venue["latitude"]))+str(int(venue["longitude"]))
        
        if idd not in my_nodesdict:
            
            node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : float(1),
                    "start" : date.get("datetime").isoformat(),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str(1)+"  \nstart : "+str(date.get("datetime").isoformat())+"  \nend : "+str((date.get("datetime")+timedelta(days=1)).isoformat())+"  \ncolor :"+str(tour_color)
                    }
        else:
                
                node = {
                    "id" : idd,
                    "name" : slugify(venue["name"]),
                    "lat" : float(venue["latitude"]),
                    "lng" : float(venue["longitude"]),
                    "weight" : math.sqrt(float(my_nodesdict[idd]["data"].get("weight"))+1) ,
                    "start" : my_nodesdict[idd]["data"].get("start"),
                    "end" : (date.get("datetime")+timedelta(days=1)).isoformat(),
                    "color" : tour_color,
                    "notes" : "name : "+slugify(venue["name"])+"  \nlat : "+str(float(venue["latitude"]))+"  \nlng :"+str(float(venue["longitude"]))+"  \n weight :"+str((float(my_nodesdict[idd]["data"].get("weight"))+1))+"  \nstart : "+str(my_nodesdict[idd]["data"].get("start"))+"  \nend : "+(date.get("datetime")+timedelta(days=1)).isoformat()+"  \ncolor :"+tour_color
                    
                    }
        if dateini == 0:
                dateini =1
        else:
                #print my_nodes[-1]["data"]["id"]
                #print dates[gig_indice-1]["distanceToNextGig"]
                notesE=""
                notesE+="distance : "
                try: 
                    notesE+=str(dates[gig_indice]["distanceToNextGig"])
                except KeyError:
                    notesE+="dates[gig_indice][distanceToNextGig] NOT AVAILABLE"
                notesE+=" km  \n"
                notesE+="source : "
                notesE+=str(my_nodes[-1]["data"]["id"])
                notesE+="  target : "
                notesE+=str(node["id"])
                notesE+="  \ndatesource : "
                notesE+=dates[gig_indice -1]["datetime"].isoformat()
                notesE+="  \ndatetarget : "
                notesE+=dates[gig_indice]["datetime"].isoformat()
                notesE+="  \ngroup: "
                notesE+=str(tour_indice)
                #notesE+=
                
                
                #print gig_indice
                edge = {
                    "source" : my_nodes[-1]["data"]["id"],
                    "target" : node["id"],
                    "color" :  tour_color,
                    # "notes" : "distance : "+str(round(dates[gig_indice]["distanceToNextGig"],2))+" km  \n"+"source : "+my_nodes[-1]["data"]["id"]+"  target : "+node["id"]+"  \ndatesource : "+dates[gig_indice -1]["datetime"].isoformat()+"  \ndatetarget : "+dates[gig_indice]["datetime"].isoformat()+"  \ngroup:"+str(tour_indice),
                    "notes" : notesE,
                    #"weight" : float(e["weight"]),
                    "start" : dates[gig_indice -1]["datetime"].isoformat(),
                    "end" : dates[gig_indice]["datetime"].isoformat(),
                    #"group" : str(tour_indice)
                    
                }
                #print edge
                my_edges.append({ "data" : edge })
        my_nodes.append({ "data" : node })   
        my_nodesdict[node["id"]]= { "data" : node }
        gig_indice+=1
    #print my_nodes
    #print "<=============================================>"
    #print my_nodesdict
    #print "<=============================================>"
    #print my_edges
     tour_indice +=1
        #print "coords",latitude,"/",longitude
    for key, value in my_nodesdict.iteritems():
            my_nodesdictList.append(value)
    
    #ADDING EDGES WEIGHT HERE
    
    terp= map(lambda x : str(sorted([x["data"]["source"],x["data"]["target"]])),my_edges)
    
    #print terp
    from collections import Counter
    c= Counter()
    for ter in terp:
        c[ter]+=1
    #print c
    for edge in my_edges:
        if str(sorted([edge["data"]["source"],edge["data"]["target"]])) not in terp:
            print "BUG!!!!"
        else:
            if edge["data"]["source"] == edge["data"]["target"]:
                #print c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))]
                #edge["data"]["notes"]+="  \ncounted"+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                #edge["data"]["weight"]= float(1)
                print "a"
            else:
                #edge["data"]["notes"]+="  \ncounted"+str(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])+"  times"
                #edge["data"]["weight"]= float(c[str(sorted([edge["data"]["source"],edge["data"]["target"]]))])
                print "b"
    #print my_edges
    
    with open("./bands_dicts/"+artist["name"]+".obj", 'wb') as handle:
                            pickle.dump([artist["name"]+"/BETA_0.3",my_nodesdictList,my_edges], handle, protocol=pickle.HIGHEST_PROTOCOL)    


Tiësto
a
a
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
a
a
b
b
b
b
b
a
b
b
b
a
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
a
a
a
b
b
b
b
b
b
a
b
b
a
a
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
a
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
a
b
b
b
b
b
b
b
b
b
a
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
a
b

Jazz
b
b
a
b
b
b
b
a
b
b
b
b
b
b
a
a
a
a
b
b
a
a
b
b
b
b
b
a
a
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
a
b
b
a
a
b
b
a
a
b
b
b
b
b
b
b
b
a
a
a
a
a
a
a
a
a
b
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
a
a
b
b
b
b
b
b
b
a
a
a
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
a
b
b
b
b
b
b
b
b
a
a
a
a
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
a
b
b
b
a
b
b
b
b
b
b
b
b
a
b
b
b
b
a
b
b
b
a
b
b
b
b
a
b
b
b
a
b
b
b
b
b
b
a
b
b
b
a
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
a
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
a
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
a
b
b
b
b

Ferry Corsten
b
b
a
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b
b
a
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
b
a
b


In [50]:
# credentials
TOPOGRAM_URL = "http://localhost:3000"
USER = "gregory.bahde@laposte.net"
PASSWORD = "matrix"

In [8]:
print len(my_nodesdict)

0


In [56]:
# connect to the topogram instance (pass debug=True params for more info )
topogram = TopogramAPIClient(TOPOGRAM_URL) #, debug=True)

# create a new user
try :
    topogram.create_user(USER, PASSWORD)
except ValueError:
    print "> User has already been created."

# login a new user if needed
resp_user_login = topogram.user_login(USER, PASSWORD)
print resp_user_login

assert(resp_user_login["status"] == "success")
assert(resp_user_login["status_code"] == 200)


ERROR:topogram-client:403 Unauthorized request


> User has already been created.
{u'status': u'success', 'status_code': 200, u'data': {u'authToken': u'b4oBSybcMBWI0qJY3fM1mY6Xh0ldm5_wKtblj4DIWIO', u'userId': u'DaofbRenqYpqqNmtz'}}


In [68]:
nodestop=[]
edgestop=[]
tit=[]
import glob, os

print os.getcwd()

for subdir, dirs, files in os.walk("./bands_dicts"): 
 for file in files:
  if file.endswith('.obj') == True:  
    with open(os.path.join(subdir, file), 'rb') as handle:
          temp= pickle.load( handle)
    #print 
    tit=temp[0].encode("utf-8")
    edgestop=temp[2]
    nodestop=temp[1]
    print edgestop
    while True:
        try:
            #print
            create_topogram(tit, nodestop, edgestop)
        except TypeError:
            continue
        break
        

/home/goonieb/gitrep/bandstour/5_visualizer/topogram-api-clientFOR_TOPO_FROM_BANDSTOUR
[{'data': {'end': '2010-09-17T23:00:00', 'target': 'set25-80', 'source': 'set25-80', 'color': '#7f3f3f', 'notes': 'distance : 328.599454 km  \nsource : set25-80  target : set25-80  \ndatesource : 2010-09-03T23:00:00  \ndatetarget : 2010-09-17T23:00:00  \ngroup: 0', 'start': '2010-09-03T23:00:00'}}, {'data': {'end': '2010-11-11T22:00:00', 'target': 'club-firestone28-81', 'source': 'set25-80', 'color': '#7f523f', 'notes': 'distance : 127.562425 km  \nsource : set25-80  target : club-firestone28-81  \ndatesource : 2010-11-23T19:00:00  \ndatetarget : 2010-11-11T22:00:00  \ngroup: 1', 'start': '2010-11-23T19:00:00'}}, {'data': {'end': '2010-11-12T21:00:00', 'target': 'hawaiian-village27-82', 'source': 'club-firestone28-81', 'color': '#7f523f', 'notes': 'distance : 335.02856 km  \nsource : club-firestone28-81  target : hawaiian-village27-82  \ndatesource : 2010-11-11T22:00:00  \ndatetarget : 2010-11-12T21:

ERROR:topogram-client:500 - Error : A topogram with the same name already exists


> Topogram already exists
{u'status': u'success', 'status_code': 200, u'data': {u'sharedPublic': False, u'title': u'NERVO/BETA_0.3', u'userId': u'DaofbRenqYpqqNmtz', u'_id': u'my4X72CK5ug9aKFiJ', u'slug': u'nervobeta_03', u'createdAt': u'2018-05-15T03:56:44.023Z'}, u'statusCode': 200}
topogram ID : my4X72CK5ug9aKFiJ
527 nodes deleted
{u'status': u'success', 'status_code': 200, u'data': [u'TYSpSgCp8yaqfseni', u'woEkSsWGaoXzqu2bg', u'9p5oW6xZew5fXrCxw', u'x2uoyGhmgLc9F2ibb', u'MMd9XJPkkfCrsGotk', u'vyLSZKkTCj3oTzAAi', u'Lp4ZteM4hfZcNCQvA', u'4PFH3hNqswXw7EkTv', u'H7kvQ7p7eNvcqYZKf', u'9kKfm8PAZixzpGGcq', u'KHPHWvGE7guDJsD8t', u'kW25LX3XiTyEwiByd', u'PFkGMyedqpWTiDC9p', u'y8oDXMjPTxkzKXbRG', u'wbAQ5NKinRteJxcTF', u'xTyjhJmd3RQmmGSnL', u'FHcEkP6XHoXikAypi', u'JotmvMZYQvtkKktuW', u'f9jEx6WyFqGJ2J3dy', u'HshzeDEZznjkPunQB', u'37yNKe5PSTDRp4SiA', u'nwdDz7vRXkKTEku9s', u'q3BMEfaAbEAnd8B7b', u'B2AWDEMMmHrzhuJeg', u'mZuvmxu4fvaaxQLxK', u'bWi7gcXvLAcXYNoTy', u'WBqXpMfPwu9ik2aHB', u'QmjPidJZaaXqN5

ValueError: No JSON object could be decoded